In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import time

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

# Initialize and train autoencoder
autoencoder = Autoencoder().to(device)
ae_criterion = nn.MSELoss()
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
num_epochs_ae = 5

print("Training Autoencoder...")
start_time_ae = time.time()
for epoch in range(num_epochs_ae):
    autoencoder.train()
    running_loss = 0
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)
        ae_optimizer.zero_grad()
        _, outputs = autoencoder(images)
        loss = ae_criterion(outputs, images)
        loss.backward()
        ae_optimizer.step()
        running_loss += loss.item()

        if (i + 1) % 150 == 0 or (i + 1) == len(train_loader):
            print(f"Epoch [{epoch + 1}/{num_epochs_ae}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}")
            running_loss = 0

print(f"Autoencoder Training Completed in {time.time() - start_time_ae:.2f} seconds")

# Save trained autoencoder
torch.save(autoencoder.state_dict(), 'autoencoder.pth')
print("Autoencoder saved as 'autoencoder.pth'")


100%|██████████| 170M/170M [00:13<00:00, 13.1MB/s]


Training Autoencoder...
Epoch [1/5], Step [150/391], Loss: 0.1814
Epoch [1/5], Step [300/391], Loss: 0.0814
Epoch [1/5], Step [391/391], Loss: 0.0408
Epoch [2/5], Step [150/391], Loss: 0.0581
Epoch [2/5], Step [300/391], Loss: 0.0499
Epoch [2/5], Step [391/391], Loss: 0.0275
Epoch [3/5], Step [150/391], Loss: 0.0419
Epoch [3/5], Step [300/391], Loss: 0.0385
Epoch [3/5], Step [391/391], Loss: 0.0223
Epoch [4/5], Step [150/391], Loss: 0.0349
Epoch [4/5], Step [300/391], Loss: 0.0332
Epoch [4/5], Step [391/391], Loss: 0.0195
Epoch [5/5], Step [150/391], Loss: 0.0312
Epoch [5/5], Step [300/391], Loss: 0.0306
Epoch [5/5], Step [391/391], Loss: 0.0178
Autoencoder Training Completed in 68.61 seconds
Autoencoder saved as 'autoencoder.pth'


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import time

# Define the Classifier
class Classifier(nn.Module):
    def __init__(self, encoder):
        super(Classifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 2 * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        features = self.encoder(x)
        output = self.classifier(features)
        return output

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Load trained autoencoder
autoencoder = Autoencoder().to(device)
autoencoder.load_state_dict(torch.load('autoencoder.pth'))
autoencoder.eval()

# Initialize classifier
classifier = Classifier(autoencoder.encoder).to(device)
clf_criterion = nn.CrossEntropyLoss()
clf_optimizer = optim.Adam(classifier.parameters(), lr=0.001)
num_epochs_clf = 10

print("Training Classifier...")
start_time_clf = time.time()
for epoch in range(num_epochs_clf):
    classifier.train()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        clf_optimizer.zero_grad()
        outputs = classifier(images)
        loss = clf_criterion(outputs, labels)
        loss.backward()
        clf_optimizer.step()
        total_loss += loss.item()

        if (i + 1) % 150 == 0 or (i + 1) == len(train_loader):
            print(f"Epoch [{epoch + 1}/{num_epochs_clf}], Step [{i + 1}/{len(train_loader)}], Loss: {total_loss / 100:.4f}")
            total_loss = 0

print(f"Classifier Training Completed in {time.time() - start_time_clf:.2f} seconds")

# Evaluate classifier
print("Evaluating Classifier...")
classifier.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples * 100
print(f"Classifier Accuracy: {accuracy:.2f}%")


Training Classifier...
Epoch [1/10], Step [150/391], Loss: 2.4498
Epoch [1/10], Step [300/391], Loss: 1.9161
Epoch [1/10], Step [391/391], Loss: 1.0351
Epoch [2/10], Step [150/391], Loss: 1.5178
Epoch [2/10], Step [300/391], Loss: 1.4324
Epoch [2/10], Step [391/391], Loss: 0.8352
Epoch [3/10], Step [150/391], Loss: 1.2160
Epoch [3/10], Step [300/391], Loss: 1.1818
Epoch [3/10], Step [391/391], Loss: 0.7137
Epoch [4/10], Step [150/391], Loss: 1.0053
Epoch [4/10], Step [300/391], Loss: 1.0218
Epoch [4/10], Step [391/391], Loss: 0.5951
Epoch [5/10], Step [150/391], Loss: 0.8369
Epoch [5/10], Step [300/391], Loss: 0.8731
Epoch [5/10], Step [391/391], Loss: 0.5034
Epoch [6/10], Step [150/391], Loss: 0.7257
Epoch [6/10], Step [300/391], Loss: 0.7212
Epoch [6/10], Step [391/391], Loss: 0.4467
Epoch [7/10], Step [150/391], Loss: 0.5907
Epoch [7/10], Step [300/391], Loss: 0.6134
Epoch [7/10], Step [391/391], Loss: 0.3727
Epoch [8/10], Step [150/391], Loss: 0.5034
Epoch [8/10], Step [300/391], L